# 2. PreProcessing

In this notebook, we're going to load the original csv file already described in the Introduction notebook, and look for data to clean or fix.
Finally, we'll save a new csv file, containing the preprocessed data, to be used in the next steps.

Let's import the pandas library to work on csv files and dataframes:

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("csv/Data.csv", engine='python')

Let's take a look at the data

In [3]:
df.head()

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,...,UBW,UBL,LBW,LBL,SJW,SJL,MaxW,MaxL,AvgW,AvgL
0,1,Adelaide,Australian Hardcourt Championships,3/01/2000,International,Outdoor,Hard,1st Round,3,Dosedel S.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Adelaide,Australian Hardcourt Championships,3/01/2000,International,Outdoor,Hard,1st Round,3,Enqvist T.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,Adelaide,Australian Hardcourt Championships,3/01/2000,International,Outdoor,Hard,1st Round,3,Escude N.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,Adelaide,Australian Hardcourt Championships,3/01/2000,International,Outdoor,Hard,1st Round,3,Federer R.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,Adelaide,Australian Hardcourt Championships,3/01/2000,International,Outdoor,Hard,1st Round,3,Fromberg R.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Since we're not going to use every column in the dataset, first we're going to list all the columns, and then we'll delete the unnecessary ones.

In [4]:
list(df.columns)

['ATP',
 'Location',
 'Tournament',
 'Date',
 'Series',
 'Court',
 'Surface',
 'Round',
 'Best of',
 'Winner',
 'Loser',
 'WRank',
 'LRank',
 'W1',
 'L1',
 'W2',
 'L2',
 'W3',
 'L3',
 'W4',
 'L4',
 'W5',
 'L5',
 'Wsets',
 'Lsets',
 'Comment',
 'CBW',
 'CBL',
 'GBW',
 'GBL',
 'IWW',
 'IWL',
 'SBW',
 'SBL',
 'B365W',
 'B365L',
 'B&WW',
 'B&WL',
 'EXW',
 'EXL',
 'PSW',
 'PSL',
 'WPts',
 'LPts',
 'UBW',
 'UBL',
 'LBW',
 'LBL',
 'SJW',
 'SJL',
 'MaxW',
 'MaxL',
 'AvgW',
 'AvgL']

In [5]:
df = df.drop(['ATP','Tournament', 'Location', 'Best of', 'W1', 'L1', 'W2', 'L2', 'W3', 'L3', 'W4', 'L4', 'W5', 'L5', 'Wsets', 
              'Lsets', 'Comment','CBW', 'CBL', 'GBW', 'GBL', 'IWW', 'IWL', 'SBW', 'SBL',
              'B365W', 'B365L', 'B&WW', 'B&WL', 'EXW', 'EXL', 'PSW', 'PSL', 'WPts',
              'LPts', 'UBW', 'UBL', 'LBW', 'LBL', 'SJW', 'SJL'], axis=1)

In [6]:
df.head()

,Date,Series,Court,Surface,Round,Winner,Loser,WRank,LRank,MaxW,MaxL,AvgW,AvgL
0,3/01/2000,International,Outdoor,Hard,1st Round,Dosedel S.,Ljubicic I.,63,77,NaN,NaN,NaN,NaN
1,3/01/2000,International,Outdoor,Hard,1st Round,Enqvist T.,Clement A.,5,56,NaN,NaN,NaN,NaN
2,3/01/2000,International,Outdoor,Hard,1st Round,Escude N.,Baccanello P.,40,655,NaN,NaN,NaN,NaN
3,3/01/2000,International,Outdoor,Hard,1st Round,Federer R.,Knippschild J.,65,87,NaN,NaN,NaN,NaN
4,3/01/2000,International,Outdoor,Hard,1st Round,Fromberg R.,Woodbridge T.,81,198,NaN,NaN,NaN,NaN


Now, as we saw in the Introduction notebook, the 'Date' attribute is not a date, but it has 'object' type.
Let's convert these objects into dates:

In [7]:
df['Date'] = pd.to_datetime(df['Date'])

In [8]:
df['Date']

0       2000-03-01
1       2000-03-01
2       2000-03-01
3       2000-03-01
4       2000-03-01
           ...    
46647   2016-09-23
46648   2016-09-23
46649   2016-09-24
46650   2016-09-24
46651   2016-09-25
Name: Date, Length: 46652, dtype: datetime64[ns]

Now the 'Date' column contains dates only.

Next, if we take a look at 'Series' column, we can see that it contains old-labeled tournament series (i.e. International Gold for ATP500, etc.)

In [9]:
df['Series'].unique()

array(['International', 'Grand Slam', 'International Gold', 'Masters',
       'Masters Cup', 'International Series', 'ATP250', 'ATP500',
       'Masters 1000'], dtype=object)

So, let's rename those values to make sure that each torunament category is different from the others.

In [10]:
df['Series'] = df['Series'].str.replace('International$','ATP250', regex=True)
df['Series'] = df['Series'].str.replace('International Series$','ATP250', regex=True)
df['Series'] = df['Series'].str.replace('International Gold$','ATP500', regex=True)
df['Series'] = df['Series'].str.replace('Masters$','Masters 1000', regex=True)

In [11]:
df['Series'].unique()

array(['ATP250', 'Grand Slam', 'ATP500', 'Masters 1000', 'Masters Cup'],
      dtype=object)

Now let's explore the 'WRank' and 'LRank' attributes:

In [12]:
#let's take the union of the sets of 'WRank' and 'LRank' values and visualize them
set(df["WRank"].unique()) | set(df['LRank'].unique())

{'1',
 '10',
 '100',
 '1001',
 '1004',
 '1005',
 '1008',
 '101',
 '1010',
 '1011',
 '1013',
 '1014',
 '1015',
 '1018',
 '102',
 '1023',
 '1024',
 '1028',
 '103',
 '1032',
 '1035',
 '1039',
 '104',
 '1042',
 '1044',
 '1045',
 '1049',
 '105',
 '1050',
 '1055',
 '1057',
 '1059',
 '106',
 '1061',
 '1064',
 '1065',
 '1066',
 '1067',
 '107',
 '1071',
 '1076',
 '1078',
 '108',
 '1082',
 '1084',
 '1086',
 '1088',
 '1089',
 '109',
 '1093',
 '1094',
 '1095',
 '1098',
 '11',
 '110',
 '1109',
 '111',
 '1112',
 '1116',
 '112',
 '1120',
 '1121',
 '1129',
 '113',
 '1139',
 '114',
 '1141',
 '1143',
 '1145',
 '1147',
 '1149',
 '115',
 '1154',
 '1156',
 '116',
 '1164',
 '117',
 '1171',
 '118',
 '1184',
 '1187',
 '119',
 '1194',
 '1198',
 '12',
 '120',
 '121',
 '1214',
 '1216',
 '1217',
 '122',
 '1223',
 '123',
 '1236',
 '124',
 '1245',
 '1249',
 '125',
 '1255',
 '1259',
 '126',
 '1267',
 '127',
 '1275',
 '1277',
 '128',
 '1281',
 '1285',
 '1288',
 '129',
 '1291',
 '13',
 '130',
 '131',
 '1311',
 '1312',

As we can see, the values above are not numbers, but objects, and there are values like 'NR' or 'nan' (look at the end).

Let's change the types of columns WRank and LRank from object to int.
In order to do this, we firstly change their type to 'str', we replace values like 'NR' or 'nan' with the standard value of 500, and finally we cast the types to int.

In [13]:
df["WRank"] = df["WRank"].astype('str')
df["LRank"] = df["LRank"].astype('str')

df["LRank"] = df["LRank"].str.replace("NR$","500",regex=True)
df["WRank"] = df["WRank"].str.replace("NR$","500",regex=True)
df["LRank"] = df["LRank"].str.replace("nan$","500",regex=True)
df["WRank"] = df["WRank"].str.replace("nan$","500",regex=True)

df['WRank'] = df['WRank'].astype('int')
df['LRank'] = df['LRank'].astype('int')

In [14]:
df.head()

,Date,Series,Court,Surface,Round,Winner,Loser,WRank,LRank,MaxW,MaxL,AvgW,AvgL
0,2000-03-01,ATP250,Outdoor,Hard,1st Round,Dosedel S.,Ljubicic I.,63,77,NaN,NaN,NaN,NaN
1,2000-03-01,ATP250,Outdoor,Hard,1st Round,Enqvist T.,Clement A.,5,56,NaN,NaN,NaN,NaN
2,2000-03-01,ATP250,Outdoor,Hard,1st Round,Escude N.,Baccanello P.,40,655,NaN,NaN,NaN,NaN
3,2000-03-01,ATP250,Outdoor,Hard,1st Round,Federer R.,Knippschild J.,65,87,NaN,NaN,NaN,NaN
4,2000-03-01,ATP250,Outdoor,Hard,1st Round,Fromberg R.,Woodbridge T.,81,198,NaN,NaN,NaN,NaN


Another problem with this dataset, as we stated in the Introduction notebook, is that the players are already classified as "Winner" and "Loser".
Since we will train a model to identify who's more likely to win given a list of match attributes, we want to fix this issue, otherwise the model will figure out that columns relative to the winner are more 'important' than others to detect who'll win the match.

That's why we decided to:
- change the name of the columns "Winner" and "Loser" in "Player 0" and "Player 1".
- for half of the records, we'll swap "Player 1" attributes with "Player 0" attributes and viceversa, in this way we'll distribute the winners equally between "Player 0" and "Player 1". 
- create a new column "Won" that will contain 0 if Player 0 has won the match or 1 otherwise.

In [15]:
df = df.rename({"Winner":"Player 0", "Loser":"Player 1", 
                "WRank":"Pl0_Rank", "LRank":"Pl1_Rank",
                "MaxW":"Max_Pl0", "MaxL":"Max_Pl1", "AvgW":"Avg_Pl0", "AvgL":"Avg_Pl1"
               }, axis=1)

In [16]:
for ix, row in df.iterrows():
    if ix % 2 == 0: 
        #swap half of the records
        old_p0 = row['Player 0']
        old_p1 = row['Player 1']
        old_p0_rank = row['Pl0_Rank']
        old_p1_rank = row['Pl1_Rank']
        old_max_p0 = row['Max_Pl0']
        old_max_p1 = row['Max_Pl1']
        old_avg_p0 = row['Avg_Pl0']
        old_avg_p1 = row['Avg_Pl1']

        df.at[ix,'Player 0'] = old_p1
        df.at[ix,'Player 1'] = old_p0
        df.at[ix,'Pl0_Rank'] = old_p1_rank
        df.at[ix,'Pl1_Rank'] = old_p0_rank
        df.at[ix,'Max_Pl0'] = old_max_p1
        df.at[ix,'Max_Pl1'] = old_max_p0
        df.at[ix,'Avg_Pl0'] = old_avg_p1
        df.at[ix,'Avg_Pl1'] = old_avg_p0

        df.at[ix,'Won'] = 1
    else:
        #don't swap these records, but just mark as winner Player 0
        df.at[ix,'Won'] = 0

In [17]:
df.head()

,Date,Series,Court,Surface,Round,Player 0,Player 1,Pl0_Rank,Pl1_Rank,Max_Pl0,Max_Pl1,Avg_Pl0,Avg_Pl1,Won
0,2000-03-01,ATP250,Outdoor,Hard,1st Round,Ljubicic I.,Dosedel S.,77,63,NaN,NaN,NaN,NaN,1.0
1,2000-03-01,ATP250,Outdoor,Hard,1st Round,Enqvist T.,Clement A.,5,56,NaN,NaN,NaN,NaN,0.0
2,2000-03-01,ATP250,Outdoor,Hard,1st Round,Baccanello P.,Escude N.,655,40,NaN,NaN,NaN,NaN,1.0
3,2000-03-01,ATP250,Outdoor,Hard,1st Round,Federer R.,Knippschild J.,65,87,NaN,NaN,NaN,NaN,0.0
4,2000-03-01,ATP250,Outdoor,Hard,1st Round,Woodbridge T.,Fromberg R.,198,81,NaN,NaN,NaN,NaN,1.0


Next, we are going to create dummy variables dataframe for the columns: 'Court', 'Surface', 'Round' and 'Series'.

Then we are going to add these dummy variables dataframes to the original one.

In [18]:
court_dummies = pd.get_dummies(df['Court'])
court_dummies.head()

,Indoor,Outdoor
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1


In [19]:
surface_dummies = pd.get_dummies(df['Surface'])
surface_dummies.head()

,Carpet,Clay,Grass,Hard
0,0,0,0,1
1,0,0,0,1
2,0,0,0,1
3,0,0,0,1
4,0,0,0,1


In [20]:
round_dummies = pd.get_dummies(df['Round'])
round_dummies.head()

,0th Round,1st Round,2nd Round,3rd Round,4th Round,Quarterfinals,Round Robin,Semifinals,The Final
0,0,1,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0


In [21]:
series_dummies = pd.get_dummies(df['Series'])
series_dummies.head()

,ATP250,ATP500,Grand Slam,Masters 1000,Masters Cup
0,1,0,0,0,0
1,1,0,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0


Now we're going to merge all these dataframes and append those to the original one:

In [22]:
df = pd.concat([df, court_dummies, surface_dummies, series_dummies, round_dummies], axis=1)
df.head()

,Date,Series,Court,Surface,Round,Player 0,Player 1,Pl0_Rank,Pl1_Rank,Max_Pl0,...,Masters Cup,0th Round,1st Round,2nd Round,3rd Round,4th Round,Quarterfinals,Round Robin,Semifinals,The Final
0,2000-03-01,ATP250,Outdoor,Hard,1st Round,Ljubicic I.,Dosedel S.,77,63,NaN,...,0,0,1,0,0,0,0,0,0,0
1,2000-03-01,ATP250,Outdoor,Hard,1st Round,Enqvist T.,Clement A.,5,56,NaN,...,0,0,1,0,0,0,0,0,0,0
2,2000-03-01,ATP250,Outdoor,Hard,1st Round,Baccanello P.,Escude N.,655,40,NaN,...,0,0,1,0,0,0,0,0,0,0
3,2000-03-01,ATP250,Outdoor,Hard,1st Round,Federer R.,Knippschild J.,65,87,NaN,...,0,0,1,0,0,0,0,0,0,0
4,2000-03-01,ATP250,Outdoor,Hard,1st Round,Woodbridge T.,Fromberg R.,198,81,NaN,...,0,0,1,0,0,0,0,0,0,0


Now we're going to the delete the 'Court', 'Surface', 'Round' and 'Series' columns since we already created the respective dummy columns:

In [23]:
df.drop(['Court', 'Surface', 'Round', 'Series'], axis=1, inplace = True)

In [24]:
df.head()

,Date,Player 0,Player 1,Pl0_Rank,Pl1_Rank,Max_Pl0,Max_Pl1,Avg_Pl0,Avg_Pl1,Won,...,Masters Cup,0th Round,1st Round,2nd Round,3rd Round,4th Round,Quarterfinals,Round Robin,Semifinals,The Final
0,2000-03-01,Ljubicic I.,Dosedel S.,77,63,NaN,NaN,NaN,NaN,1.0,...,0,0,1,0,0,0,0,0,0,0
1,2000-03-01,Enqvist T.,Clement A.,5,56,NaN,NaN,NaN,NaN,0.0,...,0,0,1,0,0,0,0,0,0,0
2,2000-03-01,Baccanello P.,Escude N.,655,40,NaN,NaN,NaN,NaN,1.0,...,0,0,1,0,0,0,0,0,0,0
3,2000-03-01,Federer R.,Knippschild J.,65,87,NaN,NaN,NaN,NaN,0.0,...,0,0,1,0,0,0,0,0,0,0
4,2000-03-01,Woodbridge T.,Fromberg R.,198,81,NaN,NaN,NaN,NaN,1.0,...,0,0,1,0,0,0,0,0,0,0


Finally, the preprocessing stage is concluded!

Now, we're going to save the preprocessed dataframe in a csv file, so that the next notebook will use this dataframe directly after loading it!

In [25]:
df.to_csv("csv/Preprocessed_Data.csv", index=False)